In [20]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/itv015970/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [21]:
orders_df = spark.read \
.format("csv") \
.option("inferschema", "true") \
.option("header", "true") \
.load("/public/trendytech/datasets/windowdatamodified.csv")

In [3]:
orders_df.show()

+--------------+-------+-----------+-------------+------------+
|       country|weeknum|numinvoices|totalquantity|invoicevalue|
+--------------+-------+-----------+-------------+------------+
|         Spain|     49|          1|           67|      174.72|
|       Germany|     48|         11|         1795|      1600.0|
|     Lithuania|     48|          3|          622|     1598.06|
|       Germany|     49|         12|         1852|      1800.0|
|       Bahrain|     51|          1|           54|      205.74|
|       Iceland|     49|          1|          319|      711.79|
|         India|     51|          5|           95|       300.0|
|     Australia|     50|          2|          133|      387.95|
|         Italy|     49|          1|           -2|       -17.0|
|         India|     49|          5|         1280|      3284.1|
|         Spain|     50|          2|          400|     1049.01|
|United Kingdom|     51|        200|        28782|    75103.46|
|        Norway|     49|          1|    

In [22]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [7]:
# find the running total of invoice value

window_spec = Window.partitionBy("country").orderBy("invoicevalue").rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [8]:
orders_df.withColumn("running_total", sum("invoicevalue").over(window_spec)).show()

+-------+-------+-----------+-------------+------------+------------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|     running_total|
+-------+-------+-----------+-------------+------------+------------------+
| Sweden|     50|          3|         3714|      2646.3|            2646.3|
|Germany|     48|         11|         1795|      1600.0|            1600.0|
|Germany|     51|          5|         1103|      1600.0|            3200.0|
|Germany|     49|         12|         1852|      1800.0|            5000.0|
|Germany|     50|         15|         1973|      1800.0|            6800.0|
| France|     51|          5|          847|       500.0|             500.0|
| France|     49|          9|         2303|       500.0|            1000.0|
| France|     48|          4|         1299|       500.0|            1500.0|
| France|     50|          6|          529|      537.32|2037.3200000000002|
|Belgium|     50|          2|          285|      625.16|            625.16|
|Belgium|   

### RANK, DENSE_RANK, ROW_NUMBER

In [9]:
my_window = Window.partitionBy("country").orderBy(desc("invoicevalue"))

In [11]:
#rank

orders_df.withColumn("rank", rank().over(my_window)).show()

+-------+-------+-----------+-------------+------------+----+
|country|weeknum|numinvoices|totalquantity|invoicevalue|rank|
+-------+-------+-----------+-------------+------------+----+
| Sweden|     50|          3|         3714|      2646.3|   1|
|Germany|     49|         12|         1852|      1800.0|   1|
|Germany|     50|         15|         1973|      1800.0|   1|
|Germany|     48|         11|         1795|      1600.0|   3|
|Germany|     51|          5|         1103|      1600.0|   3|
| France|     50|          6|          529|      537.32|   1|
| France|     51|          5|          847|       500.0|   2|
| France|     49|          9|         2303|       500.0|   2|
| France|     48|          4|         1299|       500.0|   2|
|Belgium|     48|          1|          528|       800.0|   1|
|Belgium|     51|          2|          942|       800.0|   1|
|Belgium|     50|          2|          285|      625.16|   3|
|Finland|     50|          1|         1254|       892.8|   1|
|  India

In [12]:
# dense_rank
orders_df.withColumn("dense_rank", dense_rank().over(my_window)).show()

+-------+-------+-----------+-------------+------------+----------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|dense_rank|
+-------+-------+-----------+-------------+------------+----------+
| Sweden|     50|          3|         3714|      2646.3|         1|
|Germany|     49|         12|         1852|      1800.0|         1|
|Germany|     50|         15|         1973|      1800.0|         1|
|Germany|     48|         11|         1795|      1600.0|         2|
|Germany|     51|          5|         1103|      1600.0|         2|
| France|     50|          6|          529|      537.32|         1|
| France|     51|          5|          847|       500.0|         2|
| France|     49|          9|         2303|       500.0|         2|
| France|     48|          4|         1299|       500.0|         2|
|Belgium|     48|          1|          528|       800.0|         1|
|Belgium|     51|          2|          942|       800.0|         1|
|Belgium|     50|          2|          285|     

In [13]:
# row_number
orders_df.withColumn("row_number", row_number().over(my_window)).show()

+-------+-------+-----------+-------------+------------+----------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|row_number|
+-------+-------+-----------+-------------+------------+----------+
| Sweden|     50|          3|         3714|      2646.3|         1|
|Germany|     49|         12|         1852|      1800.0|         1|
|Germany|     50|         15|         1973|      1800.0|         2|
|Germany|     48|         11|         1795|      1600.0|         3|
|Germany|     51|          5|         1103|      1600.0|         4|
| France|     50|          6|          529|      537.32|         1|
| France|     51|          5|          847|       500.0|         2|
| France|     49|          9|         2303|       500.0|         3|
| France|     48|          4|         1299|       500.0|         4|
|Belgium|     48|          1|          528|       800.0|         1|
|Belgium|     51|          2|          942|       800.0|         2|
|Belgium|     50|          2|          285|     

In [14]:
# all in one

orders_df.withColumn("rank", rank().over(my_window)) \
.withColumn("dense_rank", dense_rank().over(my_window)) \
.withColumn("row_number", row_number().over(my_window)) \
.show()

+-------+-------+-----------+-------------+------------+----+----------+----------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|rank|dense_rank|row_number|
+-------+-------+-----------+-------------+------------+----+----------+----------+
| Sweden|     50|          3|         3714|      2646.3|   1|         1|         1|
|Germany|     49|         12|         1852|      1800.0|   1|         1|         1|
|Germany|     50|         15|         1973|      1800.0|   1|         1|         2|
|Germany|     48|         11|         1795|      1600.0|   3|         2|         3|
|Germany|     51|          5|         1103|      1600.0|   3|         2|         4|
| France|     50|          6|          529|      537.32|   1|         1|         1|
| France|     51|          5|          847|       500.0|   2|         2|         2|
| France|     49|          9|         2303|       500.0|   2|         2|         3|
| France|     48|          4|         1299|       500.0|   2|         2|    

### LEAD, LAG

In [23]:
#Lag

orders_df.show(5)

+---------+-------+-----------+-------------+------------+
|  country|weeknum|numinvoices|totalquantity|invoicevalue|
+---------+-------+-----------+-------------+------------+
|    Spain|     49|          1|           67|      174.72|
|  Germany|     48|         11|         1795|      1600.0|
|Lithuania|     48|          3|          622|     1598.06|
|  Germany|     49|         12|         1852|      1800.0|
|  Bahrain|     51|          1|           54|      205.74|
+---------+-------+-----------+-------------+------------+
only showing top 5 rows



In [24]:
mywindow = Window.partitionBy("country").orderBy("weeknum")

In [25]:
#lag

orders_df.withColumn("previous_week", lag("invoicevalue").over(mywindow)).show()

+-------+-------+-----------+-------------+------------+-------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|previous_week|
+-------+-------+-----------+-------------+------------+-------------+
| Sweden|     50|          3|         3714|      2646.3|         null|
|Germany|     48|         11|         1795|      1600.0|         null|
|Germany|     49|         12|         1852|      1800.0|       1600.0|
|Germany|     50|         15|         1973|      1800.0|       1800.0|
|Germany|     51|          5|         1103|      1600.0|       1800.0|
| France|     48|          4|         1299|       500.0|         null|
| France|     49|          9|         2303|       500.0|        500.0|
| France|     50|          6|          529|      537.32|        500.0|
| France|     51|          5|          847|       500.0|       537.32|
|Belgium|     48|          1|          528|       800.0|         null|
|Belgium|     50|          2|          285|      625.16|        800.0|
|Belgi

In [28]:
#lag

orders_df \
.withColumn("previous_week", lag("invoicevalue").over(mywindow)) \
.withColumn("net_value", expr("invoicevalue - previous_week")) \
.show()

+-------+-------+-----------+-------------+------------+-------------+-------------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|previous_week|          net_value|
+-------+-------+-----------+-------------+------------+-------------+-------------------+
| Sweden|     50|          3|         3714|      2646.3|         null|               null|
|Germany|     48|         11|         1795|      1600.0|         null|               null|
|Germany|     49|         12|         1852|      1800.0|       1600.0|              200.0|
|Germany|     50|         15|         1973|      1800.0|       1800.0|                0.0|
|Germany|     51|          5|         1103|      1600.0|       1800.0|             -200.0|
| France|     48|          4|         1299|       500.0|         null|               null|
| France|     49|          9|         2303|       500.0|        500.0|                0.0|
| France|     50|          6|          529|      537.32|        500.0|  37.32000000000005|

In [29]:
#lead

orders_df.withColumn("next_week", lead("invoicevalue").over(mywindow)).show()

+-------+-------+-----------+-------------+------------+---------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|next_week|
+-------+-------+-----------+-------------+------------+---------+
| Sweden|     50|          3|         3714|      2646.3|     null|
|Germany|     48|         11|         1795|      1600.0|   1800.0|
|Germany|     49|         12|         1852|      1800.0|   1800.0|
|Germany|     50|         15|         1973|      1800.0|   1600.0|
|Germany|     51|          5|         1103|      1600.0|     null|
| France|     48|          4|         1299|       500.0|    500.0|
| France|     49|          9|         2303|       500.0|   537.32|
| France|     50|          6|          529|      537.32|    500.0|
| France|     51|          5|          847|       500.0|     null|
|Belgium|     48|          1|          528|       800.0|   625.16|
|Belgium|     50|          2|          285|      625.16|    800.0|
|Belgium|     51|          2|          942|       800.0|     n